In [1]:
import ast
import glob
import json
import os
import re
import shutil
from random import sample

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision
from IPython.display import clear_output
from torchvision.ops import box_iou

In [2]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"
SEED = 37

In [3]:
with open("/app/_data/sequences.json", "r") as f:
    seq_dict = json.load(f)

In [4]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]
df["label_change"] = df["label"] & df["no_label"].shift(1) & df["no_label"].shift(
    2
) | df["no_label"] & df["label"].shift(1) & df["label"].shift(2)
df["sequense_change"] = df["sequence"] != df["sequence"].shift(1)
df["start_subseq"] = df["sequense_change"] | df["label_change"]
df.loc[df.index[-1], "start_subseq"] = True
df["start_subseq"].sum()
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df["start_subseq"]].index):
    df.loc[start_idx:end_idx, "subsequence_id"] = subsequence_id
    start_idx = end_idx

df["subsequence_id"] = df["subsequence_id"].astype(int)
df["subsequence_id"].nunique()

138

137

# train_test_split

## random

In [5]:
l = 0
ll = 0
for i in tqdm(range(2000)):
    n_val = np.random.randint(15, 25, 1)[0]
    val_seq = sample(df.subsequence_id.unique().tolist(), n_val)
    train_seq = list(set(df.subsequence_id.tolist()) - set(val_seq))
    l = (
        df.query("subsequence_id in @train_seq")["label"].sum()
        / df.query("subsequence_id in @val_seq")["label"].sum()
    )
    ll = (
        df.query("subsequence_id in @train_seq")["len_annotation"].sum()
        / df.query("subsequence_id in @val_seq")["len_annotation"].sum()
    )
    r = (
        df.query("subsequence_id in @val_seq and len_annotation == 0").shape[0]
        / df.query("subsequence_id in @val_seq and len_annotation != 0").shape[0]
    )
    if 8 <= l <= 10 and 8 <= ll <= 10 and 3.5 <= r <= 4.3:
        break

df.query("subsequence_id in @train_seq")[["len_annotation", "label"]].sum() / df.query(
    "subsequence_id in @val_seq"
)[["len_annotation", "label"]].sum()

df.query("subsequence_id in @val_seq and len_annotation == 0").shape[0] / df.query(
    "subsequence_id in @val_seq and len_annotation != 0"
).shape[0]

sorted(val_seq)

 39% 772/2000 [00:13<00:21, 58.40it/s]


len_annotation    8.288056
label             8.683071
dtype: float64

4.188976377952756

[10, 15, 19, 32, 59, 64, 71, 76, 86, 93, 115, 120, 125, 133, 137]

In [6]:
KFOLD = "rand"
train_idx = df.query("subsequence_id in @train_seq and len_annotation>0").index.tolist()
val_idx = df.query("subsequence_id in @val_seq").index.tolist()
train = df.loc[train_idx]
val = df.loc[val_idx]
train[["len_annotation", "label"]].sum()
val[["len_annotation", "label"]].sum()
train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

len_annotation    10617
label              4411
dtype: int64

len_annotation    1281
label              508
dtype: int64

len_annotation    8.288056
label             8.683071
dtype: float64

# GroupKFold on subsequence_id

In [7]:
# train_df = df.query('len_annotation>0').reset_index(drop=True)
# kf = GroupKFold(n_splits=10)
# list_train_ids = []
# list_val_ids= []
# for kfold, (train_idx, val_idx) in enumerate(
#     kf.split(train_df, y=train_df.len_annotation, groups=train_df.subsequence_id)
# ):
#     list_train_ids.append(train_idx)
#     list_val_ids.append(val_idx)
#     print(kfold, '\n',train_df.loc[train_idx, ["len_annotation", 'label']].sum()/ train_df.loc[val_idx, ["len_annotation", 'label']].sum())

In [8]:
# KFOLD = 6
# train_idx = list_train_ids[KFOLD]
# val_idx = list_val_ids[KFOLD]
# train = train_df.loc[train_idx]
# val = train_df.loc[val_idx]
# train[["len_annotation", "label"]].sum()
# val[["len_annotation", "label"]].sum()
# train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

## video_id

In [9]:
# VIDEO_ID = 2
# ids = f'val{VIDEO_ID}'
# train  = pd.concat(
#     [
#         df.query("video_id!=@VIDEO_ID and len_annotation!=0"),
#         df.query("video_id!=@VIDEO_ID and len_annotation==0").sample(
#             int(
#                 df.query("video_id!=@VIDEO_ID and len_annotation!=0").shape[0]
#                 * 0.07
#             )
#         ),
#     ]
# ).sample(frac = 1)
# val = df.query("video_id==@VIDEO_ID and len_annotation!=0").sample(frac = 1)

## StratifiedKFold on subsequence_id

In [10]:
# df_split = (
#     df.groupby("subsequence_id")
#     .agg({"label": "max", "len_annotation": "sum", "video_frame": "count"})
#     .astype(int)
#     .reset_index()
# )
# n_splits = 10
# y=df_split["label"]
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# for fold_id, (train_idx, val_idx) in enumerate(
#     skf.split(df_split["subsequence_id"], y=y)
# ):
#     subseq_val_idx = df_split["subsequence_id"].iloc[val_idx]
#     df.loc[df["subsequence_id"].isin(subseq_val_idx), "fold"] = fold_id

# df["fold"] = df["fold"].astype(int)
# for fold in range(10):
#     print(f"\nFold {fold}")
#     df.query("fold != @fold")[["len_annotation", "label"]].sum() / df.query(
#         "fold == @fold"
#     )[["len_annotation", "label"]].sum()

In [11]:
# KFOLD = 3
# train = pd.concat(
#     [
#         df.query("fold != @KFOLD and len_annotation!=0"),
#         df.query("fold != @KFOLD and len_annotation==0").sample(
#             int(df.query("fold != @KFOLD and len_annotation!=0").shape[0] * 0.07)
#         ),
#     ]
# ).sample(frac=1)
# val = df.query("fold == @KFOLD").sample(frac=1)
# train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

# train[["len_annotation", "label"]].sum()

# val[["len_annotation", "label"]].sum()

# train_ids = train.index.tolist()
# val_ids = val.index.tolist()

# len(train_ids), len(val_ids)

# train_img_path = df.loc[train_ids, "new_img_path"].tolist()
# val_img_path = df.loc[val_ids, "new_img_path"].tolist()
# np.savetxt(
#     f"/app/_data/train_{SEED}_{KFOLD}.txt",
#     train_img_path,
#     fmt="%s",
# )
# np.savetxt(f"/app/_data/val_{SEED}_{KFOLD}.txt", val_img_path, fmt="%s")

In [12]:
train_txt = f"/app/_data/train_{SEED}_seq_id_{KFOLD}0_f2.txt"
val_txt = train_txt.replace("train_", "val_")
data_yaml_path = f"/app/_data/yolov5_f2/data/reef_data_seq_id_{KFOLD}_f2.yaml"
data_yaml_path
val_txt

'/app/_data/yolov5_f2/data/reef_data_seq_id_rand_f2.yaml'

'/app/_data/val_37_seq_id_rand0_f2.txt'

In [13]:
train_img_path = train["new_img_path"].tolist()
val_img_path = val["new_img_path"].tolist()
np.savetxt(
    train_txt,
    train_img_path,
    fmt="%s",
)
np.savetxt(val_txt, val_img_path, fmt="%s")

## Custimize parameters

In [14]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [15]:
%%writetemplate {data_yaml_path}

train: {train_txt}  # training directory
val: {val_txt}  # validation directory

# Classes
nc: 1  # number of classes
names: ["starfish"]  # class names

In [16]:
!cat {data_yaml_path}


train: /app/_data/train_37_seq_id_rand0_f2.txt  # training directory
val: /app/_data/val_37_seq_id_rand0_f2.txt  # validation directory

# Classes
nc: 1  # number of classes
names: ["starfish"]  # class names


In [17]:
!cat /app/_data/yolov5_f2/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (effic

In [18]:
%%writetemplate /app/_data/yolov5_f2/data/hyps/hyp.custom.seq.yaml
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.01
lrf: 0.1
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.2
anchor_t: 4.0
fl_gamma: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.5
fliplr: 0.5
mosaic: 1.0
mixup: 0.5
copy_paste: 0.0

# yolov5 requirements and wandb

In [23]:
!pip install --upgrade wandb
clear_output()
import wandb

wandb.login()

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)


True

In [24]:
%cd /app/_data/yolov5_f2/
!pip install -r requirements.txt
clear_output()

# Train

In [26]:
WEIGHTS = "yolov5l6.pt"
IMG_SIZE = 2880
NAME = f"{WEIGHTS[:-3]}_{IMG_SIZE}_seq_id_{KFOLD}0_f2"
NAME

'yolov5l6_2880_seq_id_rand0_f2'

In [27]:
!python train.py --img {IMG_SIZE} \
                --batch 2\
                --epochs 80 \
                --data {data_yaml_path} \
                --weights {WEIGHTS} \
                --name {NAME} \
                --hyp data/hyps/hyp.custom.seq.yaml \
                --single-cls \
                --patience 10 \
                --workers 0

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)
train: weights=yolov5l6.pt, cfg=, data=/app/_data/yolov5_f2/data/reef_data_seq_id_rand_f2.yaml, hyp=data/hyps/hyp.custom.seq.yaml, epochs=80, batch_size=2, imgsz=2880, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, optimizer=SGD, sync_bn=False, workers=0, project=runs/train, name=yolov5l6_2880_seq_id_rand0_f2, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=10, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmu

# f2_score

In [30]:
def tp_fp_fn(gt, prediction, conf_thr):
    ious = np.arange(0.3, 0.81, 0.05)
    TP, FP, FN = (
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
    )
    prediction = prediction[prediction[:, 4] > conf_thr]
    bboxes = prediction[:, :4].astype("int")
    bboxes[:, 0] = bboxes[:, 0] - bboxes[:, 2] / 2
    bboxes[:, 1] = bboxes[:, 1] - bboxes[:, 3] / 2
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    if bboxes.size != 0:
        if gt.size == 0:
            fp = bboxes.shape[0]
            FP = np.full(ious.shape[0], fp, "int16")
        else:
            iou_matrix = box_iou(torch.Tensor(gt), torch.Tensor(bboxes))
            for n, iou_thr in enumerate(ious):
                x = torch.where(iou_matrix >= iou_thr)
                tp = np.unique(x[0]).shape[0]
                fp = bboxes.shape[0] - tp
                fn = gt.shape[0] - tp
                TP[n] = tp
                FP[n] = fp
                FN[n] = fn
    else:
        if gt.size != 0:
            fn = gt.shape[0]
            FN = np.full(ious.shape[0], fn, "int16")
    return TP, FP, FN

In [31]:
with open("/app/f2_results.json", "r") as f:
    res_dict = json.load(f)

In [32]:
conf_thres = np.arange(0.1, 0.61, 0.01)
ious = np.arange(0.3, 0.81, 0.05)
res = np.zeros([conf_thres.shape[0], 3, ious.shape[0]])

path = f"/app/_data/yolov5_f2/runs/train/{NAME}/weights/best.pt"
IMG_SIZE = IMG_SIZE
model = torch.hub.load(
    "/app/_data/yolov5", "custom", path=path, source="local", force_reload=True
)
model.conf = 0.01
# chose validation set
df_test = val.copy()
# computing f2 score
for ix in tqdm(df_test.index.tolist()):
    img = np.array(Image.open(df_test.loc[ix, "img_path"]))
    prediction = model(img, size=IMG_SIZE, augment=True).xywh[0].cpu().numpy()
    prediction = prediction[prediction[:, 4] > 0.1]
    gt = np.array([list(x.values()) for x in df_test.loc[ix, "annotations"]])
    if gt.size:
        gt[:, 2] = gt[:, 2] + gt[:, 0]
        gt[:, 3] = gt[:, 3] + gt[:, 1]
    for n, c_th in enumerate(conf_thres):
        TP, FP, FN = tp_fp_fn(gt, prediction, c_th)
        res[n, 0, :] += TP
        res[n, 1, :] += FP
        res[n, 2, :] += FN
F2 = np.zeros(conf_thres.shape[0])
for c in range(conf_thres.shape[0]):
    TP = res[c, 0, :]
    FP = res[c, 1, :]
    FN = res[c, 2, :]
    recall = TP / (TP + FN)
    precission = TP / (TP + FP)
    f2 = 5 * precission * recall / (4 * precission + recall + 1e-16)
    F2[c] = np.mean(f2)

YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
Model Summary: 476 layers, 76118664 parameters, 0 gradients, 110.0 GFLOPs
Adding AutoShape... 
100% 2556/2556 [10:15<00:00,  4.15it/s]


In [33]:
if path not in res_dict:
    res_dict[path] = {
        IMG_SIZE: {
            "best": [
                np.round(conf_thres[np.argmax(F2)], 2),
                np.round(np.max(F2), 4),
            ],
            "all": list(np.round(F2, 4)),
        }
    }
else:
    res_dict[path][IMG_SIZE] = {
        "best": [
            np.round(conf_thres[np.argmax(F2)], 2),
            np.round(np.max(F2), 4),
        ],
        "all": list(np.round(F2, 4)),
    }

In [45]:
res_dict['/app/_data/yolov5/runs/train/yolov5l6_2880_seq_id_6/weights/best.pt']

{'2880': {'best': [0.12, 0.631],
  'all': [0.6298,
   0.6287,
   0.631,
   0.6289,
   0.6297,
   0.6288,
   0.628,
   0.6266,
   0.6249,
   0.6231,
   0.624,
   0.6242,
   0.6235,
   0.6211,
   0.6204,
   0.6149,
   0.6114,
   0.6102,
   0.6094,
   0.6063,
   0.6064,
   0.6032,
   0.6003,
   0.5969,
   0.592,
   0.5919,
   0.5902,
   0.5864,
   0.5824,
   0.5789,
   0.5763,
   0.5726,
   0.5666,
   0.5633,
   0.5583,
   0.5507,
   0.5438,
   0.5402,
   0.5362,
   0.5308,
   0.5193,
   0.5113,
   0.5037,
   0.4997,
   0.4934,
   0.4877,
   0.4784,
   0.4717,
   0.4661,
   0.4586,
   0.4463]}}

In [35]:
with open("/app/f2_results.json", "w") as f:
    json.dump(res_dict, f)